# Dataset & DataLoader

**https://blog.csdn.net/qq_36653505/article/details/83351808**

读取数据 预处理数据

## 1.两者区别

### torch.utils.data.Dataset
用于自定义数据集方法的抽象类
可以自己定义数据类继承这个抽象类
只需要定义__len__和__getitem__两个方法即可

### torch.utils.data.DataLoader
- 通过继承Dataset这个抽象类，我们可以定义需要的数据类。但通过迭代的方式来取得每个数据，很难实现batch，shuffle，或者多线程读取数据
可以通过torch.utils.data.DataLoader类来定义一个新的迭代器
将自定义的数据读取接口的输出/PyTorch已有的数据读取接口的输入按照batch size封装成Tensor
后续再包装成Bariable即可作为模型输入

## 2.Dataset和DataLoader两个类中用到的魔法方法
**\_\_len\_\_(self)**  **\_\_getitem\_\_(self)**  **\_\_iter\_\_(self)**  
- \_\_len\_\_(self) 定义当被len()函数调用时的行为(返回容器中元素的个数)
- \_\_getitem\_\_(self) 定义获取容器中指定元素的行为，相当于self[key],即允许类对象有索引操作
- \_\_iter\_\_(self) 定义迭代容器中的元素时的行为

## 2.1魔法方法 \_\_len\_\_() & \_\_getitem\_\_()的使用
\_\_len\_\_() & \_\_getitem\_\_()可以用于定制容器类型数据(像序列类型（如列表、元组和字符串）或映射类型（如字典）都属于容器类型数据)
- 如果容器不可变 只需要定义\_\_len\_\_()和\_\_getitem\_\_()这两个魔法方法
- 如果容器可变 除了\_\_len\_\_()和\_\_getitem\_\_()这两个魔法方法，还需要定义\_\_setitem\_\_()和\_\_delitem\_\_()两个方法  

e.g.: 编写一个不可变自定义列表，记录列表中每个元素被访问次数

In [2]:
class CountList:
    def __init__(self, *args):
        #存储列表具体值
        self.values = [x for x in args]
        #纪律列表中元素访问次数，初始值为0
        #dict.fromkeys(seq,value)用于创建一个新字典，以序列seq中元素为新字典的键，value为字典键的初始值
        self.count={}.fromkeys(range(len(self.values)),0)
    #类被len()函数调用时的行为
    def __len__(self):
        return len(self.values)
    #当类进行索引时进行的操作
    def __getitem__(self, key):
        self.count[key]+=1
        return self.values[key]
    
#实例化类
c1=CountList(1,3,5,7,9)
c2=CountLIst(2,4,6,8,10)

#调用
print(c1[1])
print(c2[1])
print(c1[1]+c2[3])
print(c1.count)
print(c2.count)

3
4
11
{0: 0, 1: 2, 2: 0, 3: 0, 4: 0}
{0: 0, 1: 1, 2: 0, 3: 1, 4: 0}


## 2.2魔法方法__iter__()的使用
在Python中构造迭代器时，需要定义\_\_iter\_\_()方法
提供迭代方法的容器称为**迭代器**(序列(列表、元组、字符串)、字典)
实现迭代器的魔法方法有两个：
- \_\_iter\_\_()  
这个方法实际上是返回迭代器本身
- \_\_next\_\_()  
该方法决定了迭代规则  \

e.g.: 定义斐波那契数列类，其实例每次顺序返回斐波那契数列的元素

In [3]:
class Fibs:
    def __init__(self, n=20):
        self.a=0
        self.b=1
        self.n=n
    
    def __iter__(self):
        return self
    
    def __next__(self):
        self.a,self.b=self.b,self.a+self.b
        if self.a>self.n:
            raise StopIteration
        return self.a

#实例化
fib=Fibs()

#调用
for i in fib:
    print(i)

1
1
2
3
5
8
13


## Dataset类

一个用来表示数据集的抽象类，其他所有的数据集都应该是这个类的子类，并且需要重写\_\_len\_\_和\_\_getitem\_\_。

- 源码  

In [4]:
class Dataset(object):  
    """An abstract class representing a Dataset.  

    All other datasets should subclass it. All subclasses should override  
    ``__len__``, that provides the size of the dataset, and ``__getitem__``,  
    supporting integer indexing in range from 0 to len(self) exclusive.  
    """

    def __getitem__(self, index):  
        raise NotImplementedError  

    def __len__(self):  
        raise NotImplementedError  

    def __add__(self, other):  
        return ConcatDataset([self, other])  

## DataLoader类  
- 